<a href="https://colab.research.google.com/github/KristianMiok/Parliamentary-Discourse/blob/main/UK_Tot_Vizz_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# To be run only once
if 0 == 1:
    !pip install gensim
    !pip install PyLDAvis
    !pip install spacy
    !python -m spacy download en_core_web_sm

In [ ]:
# Access to resources
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
# Read data from file
import pandas as pd
from sklearn.utils import shuffle

# Point to the file in Google Drive
# filename='/content/gdrive/My Drive/Topic Modeling/om.csv'
#filename='/content/gdrive/My Drive/EN_HS/big_dataset.csv'
df = pd.read_csv('/content/gdrive/My Drive/Text Summarization/UK/Data/total_80.txt', sep='\t',encoding="utf-8")

In [ ]:
df.shape

(79997, 20)

In [ ]:
df.columns

Index(['ID', 'Title', 'From', 'To', 'House', 'Term', 'Session', 'Meeting',
       'Sitting', 'Agenda', 'Subcorpus', 'Speaker_role', 'Speaker_type',
       'Speaker_party', 'Speaker_party_name', 'Party_status', 'Speaker_name',
       'Speaker_gender', 'Speaker_birth', 'V2'],
      dtype='object')

In [ ]:
df['Speaker_role'].value_counts()

Regular    79997
Name: Speaker_role, dtype: int64

In [ ]:
df1=df.loc[df['Speaker_type'] == "MP"]

In [ ]:
df2=df1.loc[df1['Speaker_role'] == "Regular"]

In [ ]:
df2.shape

(79997, 20)

In [ ]:
papers=df2

In [ ]:
# Cleaning!
# Load the regular expression library
import re

# Remove punctuation
papers['paper_text_processed'] = papers['V2'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
papers['paper_text_processed'] = papers['paper_text_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
papers['paper_text_processed'].head()

0    (urgent question): to ask the secretary of sta...
1    i thank the minister for that answer i presume...
2    my lords i declare my interests in the registe...
3    25 whether she plans to exclude international ...
4    i understand that people quite often want to b...
Name: paper_text_processed, dtype: object

In [ ]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['urgent', 'question', 'to', 'ask', 'the', 'secretary', 'of', 'state', 'for', 'transport', 'if', 'he', 'will', 'make', 'statement', 'on', 'recent', 'changes', 'to', 'aviation', 'security']


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(["say","time","want","know","friend","come","see","right","member","today","thing","may","year","week","put","last","make","leave","good","day","can","speak","great","place","thank","hear","must","way","go","think","debate","lord","member","question","plan","could","would","business","year",'people',"noble","say","government","lord","support","country","deal","work","pay","many","have", "give", "take", "make", "do", "get"])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
data_words_nostops = remove_stopwords(data_words)

In [ ]:
type(data_words_nostops)

list

In [ ]:
any("want" in w for w in data_words_nostops)

False

In [ ]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB'])

print(data_lemmatized[:1][0][:30])

['statement', 'recent', 'change', 'aviation', 'security']


In [ ]:
data_lemmatized2 = remove_stopwords(data_lemmatized)

In [ ]:
print(len(stop_words))
print(len(data_lemmatized)-len(data_lemmatized2))

235
0


In [ ]:
len(data_lemmatized2)

79997

In [ ]:
any("speak" in w for w in data_lemmatized)

True

In [ ]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized2)

# Create Corpus
texts = data_lemmatized2

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]


In [ ]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=5, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [ ]:
print(lda_model)

LdaModel(num_terms=34481, num_topics=5, decay=0.5, chunksize=100)


In [ ]:
!pip install pyLDAvis==2.1.2

In [ ]:
import numpy as np
import tqdm
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word,sort_topics=False)
LDAvis_prepared

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.207848  0.073049       1        1  17.040885
1     -0.022707 -0.234707       2        1  16.154801
2     -0.104094  0.072602       3        1  18.315567
3      0.152757 -0.041623       4        1  26.250142
4      0.181891  0.130679       5        1  22.238606, topic_info=           Term          Freq         Total Category  logprob  loglift
279        bill  21546.000000  21546.000000  Default  30.0000  30.0000
20        child  14118.000000  14118.000000  Default  29.0000  29.0000
150   amendment  16470.000000  16470.000000  Default  28.0000  28.0000
2568      trade  10560.000000  10560.000000  Default  27.0000  27.0000
1497      local  16915.000000  16915.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
670    decision   3975.616681   9519.291675   Topic5  -5.5511   0.6302
139       allow   3542.184012   9167.924425   Topic5  -5.6665   0.5524
1400      issue   4012.288313  21500.004750   Topic5  -5.5419  -0.1754
33         need   4310.967768  36278.693655   Topic5  -5.4701  -0.6267
896         end   3450.213058  10880.834504   Topic5  -5.6928   0.3548

[362 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
3124      3  0.998771  accommodation
74        3  0.095701            act
74        4  0.721157            act
74        5  0.183209            act
97        3  0.999425          adult
...     ...       ...            ...
2802      5  0.005377         worker
2804      2  0.702947          world
2804      5  0.296989          world
43        2  0.194591          young
43        3  0.805269          young

[653 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5])

In [ ]:
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, strip_numeric

lda_topics = lda_model.show_topics(num_topics=5,num_words=30)
lda_topics
# topics = []
# filters = [lambda x: x.lower(), strip_punctuation, strip_numeric]

# for topic in lda_topics:
#     print(topic)
#     topics.append(preprocess_string(topic[1], filters))

[(0,
  '0.011*"scheme" + 0.011*"need" + 0.009*"job" + 0.009*"money" + 0.008*"increase" + 0.008*"sector" + 0.008*"cost" + 0.007*"local" + 0.007*"new" + 0.007*"fund" + 0.006*"financial" + 0.006*"help" + 0.006*"public" + 0.006*"company" + 0.006*"benefit" + 0.006*"pandemic" + 0.006*"measure" + 0.006*"pension" + 0.006*"crisis" + 0.005*"economy" + 0.005*"high" + 0.005*"service" + 0.005*"small" + 0.005*"provide" + 0.005*"funding" + 0.005*"use" + 0.005*"rate" + 0.004*"transport" + 0.004*"worker" + 0.004*"look"'),
 (1,
  '0.014*"trade" + 0.010*"ensure" + 0.009*"important" + 0.009*"world" + 0.008*"need" + 0.008*"future" + 0.008*"new" + 0.008*"part" + 0.008*"continue" + 0.007*"opportunity" + 0.007*"include" + 0.006*"industry" + 0.006*"food" + 0.006*"agreement" + 0.006*"standard" + 0.005*"policy" + 0.005*"area" + 0.005*"international" + 0.005*"agree" + 0.005*"sector" + 0.005*"economic" + 0.005*"free" + 0.005*"market" + 0.005*"set" + 0.004*"nation" + 0.004*"environment" + 0.004*"commitment" + 0.004

In [ ]:
pyLDAvis.save_html(LDAvis_prepared, '/content/gdrive/My Drive/Text Summarization/UK/Html/UK_tot5.html')